In [10]:
import requests
import pandas as pd


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
# Get Paris network ID
url_networks = "http://api.citybik.es/v2/networks"
response = requests.get(url_networks)
data = response.json()
paris_id = next((network['id'] for network in data['networks'] if network['location']['city'] == 'Paris'), None)

# Paris bike station details
FOURSQUARE_SEARCH_URL = "https://api.foursquare.com/v3/places/search"
url_paris = f"http://api.citybik.es/v2/networks/{paris_id}"
response_paris = requests.get(url_paris)
data_paris = response_paris.json()
bike_stations = data_paris['network']['stations']
station = bike_stations[0]
lat, lon = station['latitude'], station['longitude']  # Define lat and lon here

foursquare_params = {
    "query": "cafe,bar",
    "ll": f"{lat},{lon}",
    "open_now": "true",
    "sort":"DISTANCE",
    "radius": 1000
}

headers = {
    "Accept": "application/json",
    "Authorization": "fsq3Nw/fR0XoqtoRT5LtvGCoAEz1vkHCNj32ho57y6khuzM="
}

foursquare_response = requests.request("GET", FOURSQUARE_SEARCH_URL, params=foursquare_params, headers=headers)
foursquare_response_json = foursquare_response.json()

print(foursquare_response_json)


{'results': [{'fsq_id': '4bade735f964a52042713be3', 'categories': [{'id': 13003, 'name': 'Bar', 'short_name': 'Bar', 'plural_name': 'Bars', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}}, {'id': 13027, 'name': 'Bistro', 'short_name': 'Bistro', 'plural_name': 'Bistros', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}, {'id': 13152, 'name': 'Brasserie', 'short_name': 'Brasserie', 'plural_name': 'Brasseries', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/french_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 31, 'geocodes': {'main': {'latitude': 48.86594, 'longitude': 2.276189}, 'roof': {'latitude': 48.86594, 'longitude': 2.276189}}, 'link': '/v3/places/4bade735f964a52042713be3', 'location': {'address': '182 avenue Victor Hugo', 'admin_region': 'Île-de-France', 'country': 'FR', 'cross_street': '67 rue Spontini', 'formatted_address': '182 avenue V

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [6]:
for result in foursquare_response_json["results"]:
        chain_data = result.get("chains", [])
        if chain_data:
            chain_name = chain_data[0].get("name", "")
        else:
            # If "chains" is empty, fall back to the "name" attribute from the main part of the response
            chain_name = result.get("name", "")
        latitude = result["geocodes"]["main"]["latitude"]
        longitude = result["geocodes"]["main"]["longitude"]
        category_name = result["categories"][0]["name"]

Put your parsed results into a DataFrame

In [11]:
foursquare_data = []

foursquare_data.append([
    chain_name, latitude, longitude, category_name
])

df_foursquare = pd.DataFrame(foursquare_data, columns=['Name', 'Latitude', 'Longitude', 'Category'])


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [12]:
YELP_SEARCH_URL = "https://api.yelp.com/v3/businesses/search"

# Query Yelp API for cafes and bars near the current station
yelp_headers = {
    "Accept": "application/json",        
    "Authorization": f"Bearer {'9W3Ud31Ji8Kpg5dC3iufULCUIiMoggC9VrpbC36s-eyL1KNmUSW7YwaptH-erVWJwd5vVEH8NbZWjJJJuYlLQx97jgowRSQF0oRDlOIPPYe3kb_GWp7C5WutYD42ZXYx'}"
}
yelp_params = {
    'latitude': lat,
    'longitude': lon,
    'categories': 'cafe,bar',
    "radius": 1000, 
    'limit': 50
}

yelp_response_json = requests.get(YELP_SEARCH_URL, headers=yelp_headers, params=yelp_params).json()
print(yelp_response_json)


{'businesses': [{'id': 'WHHt_Jb8Tgidn9mW7oDnIg', 'alias': 'la-coïncidence-paris-4', 'name': 'La Coïncidence', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/JIS7KFrJpPLjxAT-Zpe4Lg/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/la-co%C3%AFncidence-paris-4?adjust_creative=FP1M5xUXQ4GT36spxFeN3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=FP1M5xUXQ4GT36spxFeN3A', 'review_count': 511, 'categories': [{'alias': 'french', 'title': 'French'}], 'rating': 4.5, 'coordinates': {'latitude': 48.868105, 'longitude': 2.284365}, 'transactions': [], 'price': '€€', 'location': {'address1': '15 rue Mesnil', 'address2': '', 'address3': '', 'city': 'Paris', 'zip_code': '75116', 'country': 'FR', 'state': '75', 'display_address': ['15 rue Mesnil', '75116 Paris', 'France']}, 'phone': '+33147559644', 'display_phone': '+33 1 47 55 96 44', 'distance': 674.5838847008364}, {'id': '9Dg5IhmhqeQhriJfMeNZ3g', 'alias': 'le-poincaré-paris-2', 'name': 'Le Poincaré', 'image_ur

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [15]:
yelp_data = []

# Parse the Yelp API response and extract relevant data
for business in yelp_response_json.get('businesses', []):
    yelp_data.append([
        business['name'],
        business['coordinates']['latitude'],
        business['coordinates']['longitude'],
        business['review_count'],
        business['rating'],
        business.get('categories', [{}])[0].get('title', '')
    ])

Put your parsed results into a DataFrame

In [16]:
df_yelp = pd.DataFrame(yelp_data, columns=['Name', 'Latitude', 'Longitude', 'Review Count', 'Rating', 'Category'])

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The Yelp API offered several valuable features during our data analysis. For instance, it provided users with valuable information like ratings, review counts, and pricing details, making it a highly beneficial resource. Nonetheless, a drawback we encountered was its limited number of API requests, which became quite inconvenient when we needed to fetch data frequently or in large volumes.

Get the top 10 restaurants according to their rating

In [22]:
# Get the indices of the top 10 largest values
top_indices = df_yelp["Rating"].nlargest(10).index

# Extract the rows with those indices
top_10_rows = df_yelp.loc[top_indices]

# Print the top 10 rows
print(top_10_rows)

                                  Name   Latitude  Longitude  Review Count  \
48                             Sorella  48.870090   2.283070             6   
49                          L'Archeste  48.861790   2.278110             5   
0                       La Coïncidence  48.868105   2.284365           511   
2   Crêperie Framboise Passy-Trocadéro  48.858829   2.285040            65   
3                           L'Astrance  48.858440   2.287340            64   
4                             Sgabetti  48.865831   2.283666            40   
9                            Hiramatsu  48.865070   2.287510            32   
10                    Mokus l'Ecureuil  48.863964   2.287651            83   
12                        Le Pergolèse  48.874390   2.282890            48   
15                              Baāgaā  48.865169   2.287366            34   

    Rating   Category  
48     5.0    Italian  
49     5.0     French  
0      4.5     French  
2      4.5  Creperies  
3      4.5     French